# FAISS 인덱스 제작
이 노트북은 단계별로 이미지를 생성/불러와 OCR 모델로 벡터를 추출하고 FAISS 인덱스를 만드는 예시입니다.
각 코드 셀을 순서대로 실행하세요.

In [ ]:
%pip install numpy pillow tensorflow faiss-cpu

In [ ]:
# 라이브러리 임포트 및 스크립트 불러오기
import os
import numpy as np
import tensorflow as tf
print('모듈 로드 완료')

## 1) 이미지 텐서 생성 또는 불러오기
- 기존 `.npy` 텐서가 있으면 해당 파일을 사용하세요.
- 없으면 코드포인트 범위를 지정해 폰트로 렌더링하여 생성합니다.

In [ ]:
# 텐서 생성/불러기 설정 예시
TENSORS_PATH = None  # 예: './unicode_tensors.npy'
LABELS_PATH = None   # 예: './unicode_labels.npy'
START_CP = 44032     # 한글 시작: 0xAC00
END_CP = 55204       # 한글 끝+1: 0xD7A4 -> 55204
FONT_PATH = './NotoSansKR-Medium.ttf'  # 폰트 경로 (없으면 기본 폰트 사용)

# 텐서 로드 또는 생성 (실행하면 X, labels 변수가 생성됩니다)
X, labels = bf.load_or_generate_tensors(TENSORS_PATH, LABELS_PATH, START_CP, END_CP, FONT_PATH)
print('이미지 텐서 수:', X.shape, '레이블 수:', labels.shape)
# 필요하면 파일로 저장
np.save('unicode_tensors.npy', X)
np.save('unicode_labels.npy', labels)
print('텐서와 레이블 저장 완료')

## 2) OCR 모델 로드
- 학습한 `final.keras` 모델 파일 경로를 지정하세요.

In [ ]:
#OCR 모델 로드
MODEL_PATH = "./final.keras"

L_SLICE = slice(0, 19)
V_SLICE = slice(19, 40)
T_SLICE = slice(40, 68)

def hangul_loss(y_true, logits):
    cho  = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, L_SLICE], logits=logits[:, L_SLICE])
    jung = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, V_SLICE], logits=logits[:, V_SLICE])
    jong = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, T_SLICE], logits=logits[:, T_SLICE])
    return cho + jung + jong

def acc_first(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, L_SLICE], -1),
                                           tf.argmax(y_pred[:, L_SLICE], -1)), tf.float32))
def acc_middle(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, V_SLICE], -1),
                                           tf.argmax(y_pred[:, V_SLICE], -1)), tf.float32))
def acc_last(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, T_SLICE], -1),
                                           tf.argmax(y_pred[:, T_SLICE], -1)), tf.float32))
def acc_joint(y_true, y_pred):
    l_ok = tf.equal(tf.argmax(y_true[:, L_SLICE], -1), tf.argmax(y_pred[:, L_SLICE], -1))
    v_ok = tf.equal(tf.argmax(y_true[:, V_SLICE], -1), tf.argmax(y_pred[:, V_SLICE], -1))
    t_ok = tf.equal(tf.argmax(y_true[:, T_SLICE], -1), tf.argmax(y_pred[:, T_SLICE], -1))
    return tf.reduce_mean(tf.cast(l_ok & v_ok & t_ok, tf.float32))

# --- 모델 로드 ---
model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        "hangul_loss": hangul_loss,
        "acc_first": acc_first,
        "acc_middle": acc_middle,
        "acc_last": acc_last,
        "acc_joint": acc_joint,
    }
)



## 3) 벡터 추출
- OCR 모델 출력(예: 68차원)을 그대로 사용하거나, 필요하면 인코더를 통해 차원 축소합니다.

In [ ]:
# 벡터 추출 예시
BATCH_SIZE = 256
encoder_model_path = None  # 예: './best_simple_encoder_for_faiss.h5'
encoder = None
if encoder_model_path:
    encoder = tf.keras.models.load_model(encoder_model_path)

vectors = bf.extract_vectors(model, X, batch_size=BATCH_SIZE, encoder=encoder)
print('벡터 형상:', vectors.shape)
np.save('unicode_vectors.npy', vectors)
print('벡터 저장: unicode_vectors.npy')

## 4) FAISS 인덱스 생성 및 저장
- 데이터 수에 따라 Flat 또는 IVF 인덱스를 자동으로 선택합니다.
- `nprobe`는 검색시 탐색할 클러스터 개수로, 정확도/속도 조정을 위해 변경 가능합니다.

In [ ]:
# FAISS 인덱스 빌드
OUT_INDEX = 'unicode_faiss_index.faiss'
OUT_MAP = 'faiss_unicode_map.npy'
NPROBE = 20

bf.build_index(vectors, labels, OUT_INDEX, nprobe=NPROBE)
np.save(OUT_MAP, labels)
print('인덱스 및 매핑 저장 완료:', OUT_INDEX, OUT_MAP)

## 5) 간단 검색 예시
- 생성한 FAISS 인덱스와 매핑을 불러와 샘플 문자로 검색해봅니다.

In [ ]:
import faiss

index = faiss.read_index(OUT_INDEX)
u_map = np.load(OUT_MAP)

# 예시 입력 문자
query_char = '일'
q_tensor = bf.render_char_to_tensor(query_char, font_path=FONT_PATH)
q_tensor = np.expand_dims(q_tensor, 0).astype(np.float32)
q_vec = model.predict(q_tensor)
if encoder is not None:
    q_vec = encoder.predict(q_vec)

D, I = index.search(q_vec, 10)
print('Top results:')
for rank, (idx, dist) in enumerate(zip(I[0], D[0])):
    if idx >= 0:
        print(f"{rank+1}. codepoint={u_map[idx]} char={chr(int(u_map[idx]))} dist={dist}")